In [1]:
# Change directory to the root folder of Dispa-SET:
import os
import importlib
os.chdir('..')
import numpy as np
import pandas as pd
import datetime as dt
from dispaset.postprocessing import postprocessing as post
importlib.reload(post)
%matplotlib notebook

# Import dispaset
import dispaset as ds

In [2]:
# Load the configuration file
config = ds.load_config_excel('Simulations/ConfigGCC.xlsx')

[INFO    ] (load_config_excel): Using config file Simulations/ConfigGCC.xlsx to build the simulation environment
[INFO    ] (load_config_excel): Using /Users/abdullahalawad/Dropbox (MIT)/CCES/2017/Energy/Regional trade/UC&ED/dispaset/PycharmProjects/UC Model/Simulations/simulation_GCC as simulation folder


In [3]:
config['SimulationDirectory'] = u'Simulations/Test7'
config['StartDate'] = (2016, 7, 11, 0, 0, 0)      # '1/1/16'  (2016, 1, 1, 0, 0, 0)
config['StopDate'] = (2016, 7, 17, 0, 0, 0)      # '12/31/16'  (2016, 12, 31, 0, 0, 0)
config['HorizonLength'] = 7
config['LookAhead'] = 1
config['SimulationType'] = u'Integer clustering'                   # simulation type (LP or MILP) and clustering type (Integer or LP clustering)
config['ReserveCalculation'] = u'Generic'
config['AllowCurtailment'] = 1                                     #1 for True & 0 for False
config['GAMS_ModelCode'] = 'GCC_virtual_connections'               #'Standard', 'GCC_virtual_connections', 'GCC_substation_nodes', 'GCC_isolated'

In [4]:
config['PriceOfCrudeOil'] = None 
config['PriceOfDiesel'] = None 
config['PriceOfGas'] = None 
config['PriceOfHFO'] = None 

#config['PriceOfCrudeOil'] = u'Database/FuelPrices/Oil/2016_int.csv'
#config['PriceOfDiesel'] = u'Database/FuelPrices/Diesel/2016_int.csv'
#config['PriceOfGas'] = u'Database/FuelPrices/Gas/2016_int.csv'
#config['PriceOfHFO'] = u'Database/FuelPrices/HFO/2016_int.csv'

In [5]:
config['PriceOfCrudeOil 2'] = None 
config['PriceOfDiesel 2'] = None 
config['PriceOfGas 2'] = None 
config['PriceOfHFO 2'] = None 

#config['PriceOfCrudeOil 2'] = u'Database/FuelPrices/Oil/2016_sub.csv'
#config['PriceOfDiesel 2'] = u'Database/FuelPrices/Diesel/2016_sub.csv'
#config['PriceOfGas 2'] = u'Database/FuelPrices/Gas/2016_sub.csv'
#config['PriceOfHFO 2'] = u'Database/FuelPrices/HFO/2016_sub.csv'

In [6]:
# Build the simulation environment:
SimData, FuelPrices, FuelPrices2 = ds.build_simulation(config, LocalSubsidyMultiplier=1, ExportCostMultiplier=1)

[INFO    ] (build_simulation): New build started. DispaSET version: b'v2.3-7-g84905e4'
[INFO    ] (NodeBasedTable): No data file found for the table LoadShedding. Using default value 0.0
[INFO    ] (NodeBasedTable): No data file found for the table CostLoadShedding. Using default value 1000.0
[INFO    ] (check_units): The columns "Nunits" is not present in the power plant database. A value of one will be assumed by default
[WARNING ] (build_simulation): No data file or default value found for PriceOfBlackCoal 2 in the zone UAE. Assuming zero marginal price!
[WARNING ] (interconnections): Zone SA_NWOA does not appear to be connected to any other zone in the NTC table. It should be simulated in isolation
[INFO    ] (clustering): Clustered 1018 original units into 67 new units
[INFO    ] (build_simulation): Time to cluster power plants: 38.886865854263306s
[WARNING ] (build_simulation): TimeUpMinimum in the power plant data has been rounded to the nearest integer value
[WARNING ] (build_s

In [7]:
#config['GAMS_folder'] = '/Applications//GAMS26.1/sysdir'
#config['GAMS_folder'] = 'C:\GAMS\win64\24.7'

In [8]:
# Solve using GAMS:
r = ds.solve_GAMS(config['SimulationDirectory'], config['GAMS_folder'])

[INFO    ] (solve_GAMS): Using the low-level gams api
[WARNING ] (get_gams_path): The provided path for GAMS ( ) does not exist. Trying to locate...
[INFO    ] (get_gams_path): Detected /Applications//GAMS27.2/sysdir as GAMS path on this computer
[INFO    ] (solve_low_level): Completed simulation in 115.37 seconds


##  4 scenarios (all with export dispatch with international prices):
#### 1. local dispatch with subsidized prices & export dispatch with international prices
#### 2. local dispatch with international prices & export dispatch with international prices
#### 3. local dispatch with subsidized prices & no export
#### 4. local international with subsidized prices & no export

In [ ]:
# Load the configuration file
config = ds.load_config_excel('Simulations/ConfigGCC.xlsx')

In [ ]:
config['SimulationDirectory'] = u'Simulations/simulation_GCC_sub'
config['StartDate'] = (2016, 7, 11, 0, 0, 0)                       # '1/1/16'  (2016, 1, 1, 0, 0, 0)
config['StopDate'] = (2016, 7, 17, 0, 0, 0)                        # '12/31/16'  (2016, 12, 31, 0, 0, 0)
config['HorizonLength'] = 7
config['LookAhead'] = 1
config['SimulationType'] = u'Integer clustering'                   # simulation type (LP or MILP) and clustering type (Integer or LP clustering)
config['ReserveCalculation'] = u'Generic'
config['AllowCurtailment'] = 1                                     #1 for True & 0 for False
config['GAMS_ModelCode'] = 'GCC_virtual_connections'               #'Standard', 'GCC_virtual_connections', 'GCC_substation_nodes', 'GCC_isolated'

config['PriceOfCrudeOil'] = u'Database/FuelPrices/Oil/2016_sub.csv'
config['PriceOfDiesel'] = u'Database/FuelPrices/Diesel/2016_sub.csv'
config['PriceOfGas'] = u'Database/FuelPrices/Gas/2016_sub.csv'
config['PriceOfHFO'] = u'Database/FuelPrices/HFO/2016_sub.csv'

In [ ]:
# Build the simulation environment:
SimData, FuelPrices = ds.build_simulation(config2)

In [ ]:
# Solve using GAMS:
r = ds.solve_GAMS(config2['SimulationDirectory'], config2['GAMS_folder'])

In [ ]:
# Load the configuration file
config2 = ds.load_config_excel('Simulations/ConfigGCC.xlsx')

In [ ]:
config2['SimulationDirectory'] = u'Simulations/simulation_GCC_int'
config2['StartDate'] = (2016, 7, 11, 0, 0, 0)                       # '1/1/16'  (2016, 1, 1, 0, 0, 0)
config2['StopDate'] = (2016, 7, 17, 0, 0, 0)                        # '12/31/16'  (2016, 12, 31, 0, 0, 0)
config2['HorizonLength'] = 7
config2['LookAhead'] = 1
config2['SimulationType'] = u'Integer clustering'                   # simulation type (LP or MILP) and clustering type (Integer or LP clustering)
config2['ReserveCalculation'] = u'Generic'
config2['AllowCurtailment'] = 1                                     #1 for True & 0 for False
config2['GAMS_ModelCode'] = 'GCC_virtual_connections'               #'Standard', 'GCC_virtual_connections', 'GCC_substation_nodes', 'GCC_isolated'

config2['PriceOfCrudeOil'] = u'Database/FuelPrices/Oil/2016_int.csv'
config2['PriceOfDiesel'] = u'Database/FuelPrices/Diesel/2016_int.csv'
config2['PriceOfGas'] = u'Database/FuelPrices/Gas/2016_int.csv'
config2['PriceOfHFO'] = u'Database/FuelPrices/HFO/2016_int.csv'

In [ ]:
# Build the simulation environment:
SimData2, FuelPrices2 = ds.build_simulation(config2)

In [ ]:
# Solve using GAMS:
r2 = ds.solve_GAMS(config2['SimulationDirectory'], config2['GAMS_folder'])

In [ ]:
# Load the configuration file
config3 = ds.load_config_excel('Simulations/ConfigGCC.xlsx')

In [ ]:
config3['SimulationDirectory'] = u'Simulations/simulation_GCC_sub_isolated'
config3['StartDate'] = (2016, 7, 11, 0, 0, 0)                       # '1/1/16'  (2016, 1, 1, 0, 0, 0)
config3['StopDate'] = (2016, 7, 17, 0, 0, 0)                        # '12/31/16'  (2016, 12, 31, 0, 0, 0)
config3['HorizonLength'] = 7
config3['LookAhead'] = 1
config3['SimulationType'] = u'Integer clustering'                   # simulation type (LP or MILP) and clustering type (Integer or LP clustering)
config3['ReserveCalculation'] = u'Generic'
config3['AllowCurtailment'] = 1                                     #1 for True & 0 for False
config3['GAMS_ModelCode'] = 'GCC_virtual_connections'               #'Standard', 'GCC_virtual_connections', 'GCC_substation_nodes', 'GCC_isolated'

config3['PriceOfCrudeOil'] = u'Database/FuelPrices/Oil/2016_sub.csv'
config3['PriceOfDiesel'] = u'Database/FuelPrices/Diesel/2016_sub.csv'
config3['PriceOfGas'] = u'Database/FuelPrices/Gas/2016_sub.csv'
config3['PriceOfHFO'] = u'Database/FuelPrices/HFO/2016_sub.csv'

In [ ]:
# Build the simulation environment:
SimData3, FuelPrices3 = ds.build_simulation(config3)

In [ ]:
# Solve using GAMS:
r3 = ds.solve_GAMS(config3['SimulationDirectory'], config3['GAMS_folder'])

In [ ]:
# Load the configuration file
config4 = ds.load_config_excel('Simulations/ConfigGCC.xlsx')

In [ ]:
config4['SimulationDirectory'] = u'Simulations/simulation_GCC_int_isolated'
config4['StartDate'] = (2016, 7, 11, 0, 0, 0)                       # '1/1/16'  (2016, 1, 1, 0, 0, 0)
config4['StopDate'] = (2016, 7, 17, 0, 0, 0)                        # '12/31/16'  (2016, 12, 31, 0, 0, 0)
config4['HorizonLength'] = 7
config4['LookAhead'] = 1
config4['SimulationType'] = u'Integer clustering'                   # simulation type (LP or MILP) and clustering type (Integer or LP clustering)
config4['ReserveCalculation'] = u'Generic'
config4['AllowCurtailment'] = 1                                     #1 for True & 0 for False
config4['GAMS_ModelCode'] = 'GCC_virtual_connections'               #'Standard', 'GCC_virtual_connections', 'GCC_substation_nodes', 'GCC_isolated'

config4['PriceOfCrudeOil'] = u'Database/FuelPrices/Oil/2016_int.csv'
config4['PriceOfDiesel'] = u'Database/FuelPrices/Diesel/2016_int.csv'
config4['PriceOfGas'] = u'Database/FuelPrices/Gas/2016_int.csv'
config4['PriceOfHFO'] = u'Database/FuelPrices/HFO/2016_int.csv'

In [ ]:
# Build the simulation environment:
SimData4, FuelPrices4 = ds.build_simulation(config4)

In [ ]:
# Solve using GAMS:
r4 = ds.solve_GAMS(config4['SimulationDirectory'], config4['GAMS_folder'])

SimData = {'sets': sets, 'parameters': parameters, 'config': config, 'units': Plants_merged, 'version': dispa_version}
.......
sets_param = {}
    sets_param['AvailabilityFactor']
.
.
parameters = {}
for var in sets_param:
        parameters[var] = define_parameter(sets_param[var], sets, value=0)
.
.
.
sets = {}
sets['h'] = [str(x + 1) for x in range(Nhours_long)]
sets['z'] =
.
.
.
plants = load_csv(config['PowerPlantData'])
Plants_merged, mapping = clustering(plants, method='LP')
.
.
.
param config: Dictionary with all the configuration fields loaded from the excel file. Output of the 'LoadConfig' function.
.
.
.
dispa_version = str(get_git_revision_tag())

*******&&&&&&
parameters['Config'] = {'sets': ['x_config', 'y_config'], 'val': values}
sets['x_config'] = ['FirstDay', 'LastDay', 'RollingHorizon Length', 'RollingHorizon LookAhead']
sets['y_config'] = ['year', 'month', 'day']
parameters['PowerInitial']['val'] = Plants_merged['InitialPower'].values
parameters['PowerInitial']['val'][i] = (Plants_merged['PartLoadMin'][i] + 1) / 2 * Plants_merged['PowerCapacity'][i]
parameters['Location']['val'][:, i] = (Plants_merged['Zone'] == config['countries'][i]).values
parameters['Fuel']['val'][unit, idx] = True
parameters['Technology']['val'][unit, idx] = True
parameters['Reserve'] = {'sets': sets_param['Reserve'], 'val': values}
parameters['LineNode'] = incidence_matrix(sets, 'l', parameters, 'LineNode')
parameters['FlowMinimum']['val'][i, :] = Inter_RoW[l]
parameters['CostVariable']['val'][unit, :] = FuelPrices[zone][FuelEntries[FuelEntry]] / Plants_merged['Efficiency'][unit] + Plants_merged['EmissionRate'][unit] * FuelPrices[zone]['PriceOfCO2']

